In [1]:
import numpy as np
import pandas as pd

In [2]:
pm=pd.read_csv('Poems.csv')

pm.head()

,title,poem,writter,label
0,সেই ছেলে হবে কবে,আমাদের দেশে সেই ছেলে হবে কবে যারা কোথায় না বড় ...,মুহম্মদ জাফর ইকবাল,Policy
1,সন্ত্রাসীদের প্রথম পাঠ,অ অজ্ঞান পার্টি অজ্ঞান পার্টি আসছে ঐ আ আগুন আগ...,মুহম্মদ জাফর ইকবাল,Policy
2,পারো তো ধর্ষণ করো,"আর ধর্ষিতা হয়ো না, আর না আর যেন কোনও দুঃসংবাদ ...",তসলিমা নাসরিন,Policy
3,ফেস অফ,মেয়েটি আসছে মুখটি পোড়া মুখটি এখন আর মুখের মত দ...,তসলিমা নাসরিন,Policy
4,মন উঠো,"মন তুমি ওঠো, ওঠো তুমি, তুমি ওঠো মন, মন মন মন ও...",তসলিমা নাসরিন,Policy


In [3]:
pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2207 entries, 0 to 2206
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2207 non-null   object
 1   poem     2207 non-null   object
 2   writter  2207 non-null   object
 3   label    2207 non-null   object
dtypes: object(4)
memory usage: 69.1+ KB


In [4]:
pm.describe()

,title,poem,writter,label
count,2207,2207,2207,2207
unique,1918,2020,53,11
top,তুমি,বল বীর - বল উন্নত মম শির! শির নেহারি’ আমারি নত...,রবীন্দ্রনাথ ঠাকুর,Miscellaneous
freq,5,4,250,811


In [5]:
pm.label.unique()

array(['Policy', 'Religious', 'War', 'Humanity', 'Nature',
       'Miscellaneous', 'Love', 'Metaphor', 'Separation', 'Children',
       'Patriotic'], dtype=object)

In [6]:
pm_name=['Policy', 'Religious', 'War', 'Humanity', 'Nature', 'Separation', 'Children', 'Patriotic', 'Metaphor']

def drop_column(ex):
  omg=pm[(pm['label'] == ex)].index
  pm.drop(omg, inplace=True)

for x in range(len(pm_name)):
  drop_column(pm_name[x])


In [7]:
pm.label.unique()

array(['Miscellaneous', 'Love'], dtype=object)

In [8]:
pm.describe()

,title,poem,writter,label
count,1311,1311,1311,1311
unique,1275,1307,36,2
top,তুমি,"ভাবছি, ঘুরে দাঁড়ানোই ভালো। এতো কালো মেখেছি দু...",মহাদেব সাহা,Miscellaneous
freq,5,2,170,811


In [9]:
pm.head()

,title,poem,writter,label
295,না,কিছুতেই রাজি নয় বল বাবাজি নারাজি নারাজি নারাজ...,মুহম্মদ জাফর ইকবাল,Miscellaneous
296,তুই কোথায় শেফালি,আমার খুব দেখতে ইচ্ছে করছে তুই কোথায় আমার খুব ...,তসলিমা নাসরিন,Miscellaneous
297,না-থাকা,একটি ভীষণ না-থাকাকে সঙ্গে নিয়ে আমি প্রতি রাত্...,তসলিমা নাসরিন,Miscellaneous
298,কলকাতা তুই তোর হৃদয়,"সবখানেই পুঁজিবাদের হাতি হাঁটছে, সবখানেই সাম্রা...",তসলিমা নাসরিন,Miscellaneous
299,শেষ পর্যন্ত,"না, কলকাতা শেষপর্যন্ত তুমিও আমার কোনও সমাধান ন...",তসলিমা নাসরিন,Miscellaneous


In [10]:
pm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1311 entries, 295 to 1605
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1311 non-null   object
 1   poem     1311 non-null   object
 2   writter  1311 non-null   object
 3   label    1311 non-null   object
dtypes: object(4)
memory usage: 51.2+ KB


In [11]:
import re
whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
bangla_fullstop = u"\u0964"
punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/—,¦!?;-]+"


for i in range(len(pm)):
  txt=pm.poem.values[i]

  txt = whitespace.sub(" ",txt).strip()
  txt = re.sub(punctSeq, " ", txt)
  txt = re.sub(bangla_fullstop, "",txt)
  txt = re.sub(punc, " ", txt)

  pm.poem.values[i]=txt

  

#txt = whitespace.sub(" ",txt).strip()
#txt = re.sub(punctSeq, " ", txt)
#txt = re.sub(bangla_fullstop, "",txt)
#txt = re.sub(punc, " ", txt)
#print(txt)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le=LabelEncoder()
 
pm['label']=le.fit_transform(pm['label']) 

In [14]:
pm.label.unique()

array([1, 0])

In [17]:
pm1=pm.sample(frac=1, random_state=42)

pm1.head()

,title,poem,writter,label
1415,নকশী কাঁথার মাঠ – ১২,বার রাইত তুই যা রে পোহাইয়ে বেলা গে ল সন্ধ্যা...,জসীম উদ্‌দীন,0
1367,ভিতর-বাইরে বিষম যুদ্ধ,ইচ্ছে ছিলো তোমার কাছে ঘুরতে ঘুরতে যাবোই আমার প...,শক্তি চট্টোপাধ্যায়,0
493,মাঠের সন্ধ্যা,অন্যমনে যেতে যেতে হঠাৎ যদি মাঠের মধ্যে দাঁড়াই...,নীরেন্দ্রনাথ চক্রবর্তী,1
870,সোনার তরী,গগনে গরজে মেঘ ঘন বরষা কূলে একা বসে আছি নাহি ...,রবীন্দ্রনাথ ঠাকুর,1
626,শস্যযাত্রা,তোমাকে ধরবে না এই কালো পাটকেলে কামিজে খুলে এই ...,মহাদেব সাহা,1


In [18]:
pm1.label.unique()

array([0, 1])

In [19]:
Y=pd.get_dummies(pm1.label).values

print('Shape of label tensor:',Y.shape)

Shape of label tensor: (1311, 2)


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
cnt_vec=CountVectorizer()
X=cnt_vec.fit_transform(pm1.poem.values)

print(X.shape)

(1311, 4116)


In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
tfidf=TfidfTransformer()
X=tfidf.fit_transform(X)

print(X.shape)

(1311, 4116)


In [ ]:
print(pm1.label)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X, pm1.label, test_size=0.2, random_state=42)

print('Train Set Size:', X_train.shape, Y_train.shape)
print('Test Set size:', X_test.shape, Y_test.shape)

Train Set Size: (1048, 4116) (1048,)
Test Set size: (263, 4116) (263,)


In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model=LogisticRegression()

In [29]:
penalty=['l1', 'l2', 'elasticnet', 'None']

C=np.logspace(-4,4,20)

solver=['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga']

max_iter=[100, 300, 500, 1000]


hyperparameters=dict(C=C, penalty=penalty, solver=solver, max_iter=max_iter)

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
clf=GridSearchCV(model, hyperparameters, cv=5, verbose=True, n_jobs=-1)

In [32]:
best_model=clf.fit(X_train, Y_train)

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 2230 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 5247 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 6784 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 8000 out of 8000 | elapsed:  8.7min finished


In [ ]:
best_model.best_estimator_

LogisticRegression(C=4.281332398719396, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
print(f'Accuracy : {best_model.score(X_test, Y_test):.3f}')

Accuracy : 0.829


In [34]:
Y_pred=best_model.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix

In [36]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       101
           1       0.82      0.93      0.87       162

    accuracy                           0.83       263
   macro avg       0.83      0.80      0.81       263
weighted avg       0.83      0.83      0.82       263



In [37]:
from sklearn.naive_bayes import MultinomialNB

In [38]:
modelNB=MultinomialNB()

In [39]:
mnb_alpha=[0.01, 0.0001, 0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

mnb_fit_prior=[True, False]

mnb_class_prior=[None, [0.1,0.9], [0.2, 0.8], [0.25, 0.5]]


hyperparametersNB=dict(alpha=mnb_alpha, fit_prior=mnb_fit_prior, class_prior=mnb_class_prior)

In [40]:
from sklearn.model_selection import GridSearchCV

In [41]:
clf=GridSearchCV(modelNB, hyperparametersNB, cv=5, verbose=True, n_jobs=-1)

In [42]:
best_modelNB=clf.fit(X_train, Y_train)

Fitting 5 folds for each of 104 candidates, totalling 520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 520 out of 520 | elapsed:    2.6s finished


In [43]:
best_modelNB.best_estimator_

MultinomialNB(alpha=0.3, class_prior=None, fit_prior=False)

In [44]:
print(f'Accuracy : {best_modelNB.score(X_test, Y_test):.3f}')

Accuracy : 0.856


In [45]:
Y_pred=best_modelNB.predict(X_test)

In [46]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.90      0.70      0.79       101
           1       0.84      0.95      0.89       162

    accuracy                           0.86       263
   macro avg       0.87      0.83      0.84       263
weighted avg       0.86      0.86      0.85       263



In [47]:
from sklearn.svm import SVC

In [48]:
modelSVM=SVC()

In [49]:
C=[0.001, 0.1, 1, 10, 25, 50, 100]

gamma=[1, 0.1, 0.01, 0.001, 0.0001]

kernel=['rbf', 'poly', 'sigmoid', 'linear']

hyperparametersSVM=dict(C=C, gamma=gamma, kernel=kernel)

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
clfSVM=GridSearchCV(modelSVM, hyperparametersSVM, cv=5, verbose=True, n_jobs=-1)

In [52]:
best_modelSVM=clfSVM.fit(X_train, Y_train)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   54.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed:  3.2min finished


In [53]:
best_modelSVM.best_estimator_

SVC(C=50, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [54]:
print(f'Accuracy : {best_modelSVM.score(X_test, Y_test):.3f}')

Accuracy : 0.829


In [55]:
Y_pred=best_modelSVM.predict(X_test)

In [56]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.86      0.66      0.75       101
           1       0.82      0.93      0.87       162

    accuracy                           0.83       263
   macro avg       0.84      0.80      0.81       263
weighted avg       0.83      0.83      0.82       263

